In [ ]:

#     В ЭТОМ КОДЕ:

#     + кластеризуем посты в группе ССММ (про выборы/нет, анонсы/результаты выборов)
#     + посты с анонсами выборов конвертируем в словарики (имя, курс-поток-группа, текст программы)
#     + посты с результатами выборов конвертируем в словарики со всей информацией об итогах выборов
#     + последние два пункта адаптировать для постов про магистров
#   !!! расставляем флаги предпочтений кандидатов, исходя из текста программ


In [2]:
import requests
from bs4 import Tag, NavigableString, BeautifulSoup
import re
from datetime import datetime, date, time

import pandas as pd


In [ ]:
'''
ЧТО ПОФИКСИТЬ:
- БАГ: путает блок текста в шапке репоста с блоком текста в самом репосте (пример: https://vk.com/wall-33327655_2261)
- находить посты с перечнем избранных должностей
- https://vk.com/wall-33327655_5491 - запись удалена, но код этого не видит
- https://vk.com/wall-33327655_5492 - запись не найдена, а код сломался
- https://vk.com/wall-33327655_9414 - код сломался
- были парные кандидаты, найти и расщепить
'''

# СДЕЛАТЬ ПАРСИНГ ОБСУЖДЕНИЙ В ГРУППЕ:
#     треды с кандидатами 1-3 созывов
#     треды с отчетами о собраний 1-2(3?) созывов


In [134]:
# ссылка на репост без шапки, репостнутая запись - без текста (обновление аватарки):

req_announcement = requests.get('https://vk.com/wall-33327655_9846')
html_doc = req_announcement.text
print(html_doc)

# типы постов:

#     https://vk.com/wall-33327655_2250 -- пост с тегом
#     https://vk.com/wall-33327655_2261 -- пост без тега
#     https://vk.com/wall-33327655_2255 -- post not found
#     https://vk.com/wall-33327655_2262 -- коммент под постом с тегом
#     https://vk.com/wall-33327655_3330 -- репост без шапки поста с адресом 104154624_1
#     https://vk.com/wall-33327655_8489 -- репост с шапкой поста по адресу 77014399_6331


            <!DOCTYPE html>
      <html lang="ka" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="МФК в осеннем семестре    #образование_ссмм    Запись на Межфакультетские курсы в осеннем семестре 2021/2022 учебного года откроется 3 окт" />
        <meta property="og:description" content="МФК в осеннем семестре    #образование_ссмм    Запись на Межфакультетские курсы в осеннем семестре 2021/2022 учебного года откроется 3 окт" />
        <link re

## CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK

In [137]:
'''
CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK
'''

def vk_post_info_by_link(link_vk, language):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
#     дефолтные значения
    content_type = 'no text'
    content_subtype = 'other'
    parent_link = ''
    text_lines = []
    link_year_no = 0
    link_month_no = 0
    link_day_no = 0
    link_date = date(1970, 1, 1)
    link_date_time = ''
    link_time = ''
    months_dict = {
        'russian': ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'],
        'english': ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
#         https://en.wikipedia.org/wiki/Georgian_calendar
        'georgian': ['იან', 'თებ', 'მარ', 'აპრ', 'მაი', 'ივნ', 'ივლ', 'აგვ', 'სექ', 'ოქტ', 'ნოე', 'დეკ']
    }
    months = months_dict[language]

    # ПРОВЕРЯЮ, СУЩЕСТВУЕТ ЛИ ПОСТ
    #     <div class="service_msg service_msg_error">Запись не найдена.</div>
    flg_service_msg_error = soup.find('div', attrs={'class': 'service_msg service_msg_error'})
    if flg_service_msg_error:
        link_type = 'page not found'
    else:
        link_type = 'post'
        # когда ссылка на пост: <link rel="canonical" href="https://vk.com/wall-33327655_2225" />
        # когда ссылка на коммент: <link rel="canonical" href="https://vk.com/wall-33327655_8793?reply=8800" />
        
        # ПРОВЕРЯЮ, НЕ КОММЕНТ ЛИ ЭТО
        rel_canonical = soup.find('link', attrs={'rel': 'canonical'})
        flg_rel_canonical_reply = re.findall('reply=', str(rel_canonical))
#             + ДОБАВИТЬ ПОЛЕ МАТЕРИНСКОЙ ССЫЛКИ
        if flg_rel_canonical_reply:
            link_type = 'comment'
            
        # ПРОВЕРЯЮ, НЕ РЕПОСТ ЛИ ЭТО
        flg_repost = soup.find('div', attrs={'data-copy': True})
        if flg_repost:
            link_type = 'repost'
#             <div class="wall_item single_wall_item bl_item mark_top_verified" data-copy='-77014399_6331'>
            parent_link = 'https://vk.com/wall' + flg_repost['data-copy']
    
    if link_type in ['post','repost']:
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ ПОСТА ПОСТРОЧНО    
        # <div class="wi_body">
        #     <div class="pi_text">

        wi_body = soup.find('div', attrs={'class': 'wi_body'})
        if wi_body is None:
            announcement_list = []
            result_list = []
        else:
            pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
            if pi_text is None:
                announcement_list = []
                result_list = []
            else:
                text_lines = [line for line in pi_text.stripped_strings]
                    # определить тип контента
                    # анонсы: 'Орфография и пунктуация сохранены.'
                    # результаты: 'Таким образом, выборы признаются состоявшимися'
                announcement_list = ['announcement' for line in pi_text.stripped_strings if line.find('Орфография и пунктуация сохранены.') >= 0]
                result_list = ['result' for line in pi_text.stripped_strings if line.find('Таким образом, выборы признаются состоявшимися.') >= 0]
                content_type = 'text'
                if any(election_tag in text_lines for election_tag in ['#выборы@stud_mm','#выборы_ссмм']):
                    content_type = 'election'
    
                if len(announcement_list) > 0:
                    content_subtype = 'announcement'
                elif len(result_list) > 0:
                    content_subtype = 'result'
        
        # ВЫТАСКИВАЮ ДАТУ
        # <div class="wi_info">
        #     <span class="wi_date">
        wi_info = soup.find('div', attrs={'class': 'wi_info'})
        if wi_info:
            wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
            date_list = [date for date in wi_date.stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
            link_date_time = date_list[0]
            link_time = re.findall(r'[\d]+\:[\d]+', date_list[0])[0]
        
    if link_type == 'comment':
        reply_link = link_vk[link_vk.index('_')+1:]
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ КОММЕНТА ПОСТРОЧНО    
#             <div class="ReplyItem__body">    180% голосов. Это выборы на мехмате? Это проценты чего?</div>
#             <div class="ReplyItem__date">
#             <a class="item_date" href="https://m.vk.com/wall-33327655_2254?reply=2262#reply2262" rel="noopener">22 &#4316;&#4317;&#4308;&#4315; 2014</a></div>
#         reply_body = soup.find('div', attrs={'class': 'ReplyItem__body'})
        reply_bodies = soup.find_all('div', attrs={'class': 'ReplyItem__body'})
        reply_dates = soup.find_all('a', attrs={'class': 'item_date'})
        reply_links = [ff['href'][ff['href'].index('#reply')+6:] for ff in soup.find_all('a', attrs={'class': 'item_date'})]
#         reply_no = reply_links.index(reply_link)
#         if reply_body is None:
#             announcement_list = []
#             result_list = []
#         else:
        if reply_link in reply_links:
            reply_no = reply_links.index(reply_link)
            content_type = 'text'
            text_lines = [line for line in reply_bodies[reply_no].stripped_strings]
            date_list = [date for date in reply_dates[reply_no].stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
        else:
            content_subtype = 'deleted (?)'
            
        parent_link = rel_canonical['href'][: rel_canonical['href'].index('?')]
    
    dict_link = {}
    dict_link[link_vk] = {
        'link_type': link_type, 
        'content_type': content_type,
        'content_subtype': content_subtype,
        'parent_link': parent_link,
        
        'link_date': link_date, 
        'link_year_no': link_year_no, 
        'link_month_no': link_month_no, 
        'link_day_no': link_day_no, 
        'link_date_time': link_date_time, 
        'link_time': link_time, 
        
        'list_of_lines': text_lines
    }
    
    return dict_link


In [246]:
# тест
# vk_post_info_by_link('https://vk.com/wall-33327655_5106', 'georgian') # -- пост с анонсом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2262', 'georgian') # -- коммент под постом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2263', 'georgian') # -- коммент под постом +
# vk_post_info_by_link('https://vk.com/wall-33327655_3330', 'georgian') # -- репост без шапки +
# vk_post_info_by_link('https://vk.com/wall-33327655_2261', 'georgian') # -- репост без шапки - ПОЧЕМУ ТУТ text, а не no text
# vk_post_info_by_link('https://vk.com/wall-33327655_8489', 'georgian') # -- репост с шапкой +
# vk_post_info_by_link('https://vk.com/wall-33327655_9846', 'georgian') # -- удаленный коммент (?)
# vk_post_info_by_link('https://vk.com/wall-33327655_2225', 'georgian') # -- пост с кандидатами в КОММЕНТЕ 2232
# vk_post_info_by_link('https://vk.com/wall-33327655_2232', 'georgian') # -- коммент с кандидатами

vk_post_info_by_link('https://vk.com/wall-33327655_2218', 'georgian') # -- РАЗНОЕ




{'https://vk.com/wall-33327655_2218': {'link_type': 'post',
  'content_type': 'election',
  'content_subtype': 'announcement',
  'parent_link': '',
  'link_date': datetime.date(2014, 11, 18),
  'link_year_no': 2014,
  'link_month_no': 11,
  'link_day_no': 18,
  'link_date_time': '18 ნოემ 2014  1:42',
  'link_time': '1:42',
  'list_of_lines': ['#выборы@stud_mm',
   'Напоминаем, что сегодня пройдут выборы в студсовет на двух потоках. С кандидатами с пятого курса мы уже познакомились, на очереди 4 поток 3 курса.',
   'Орфография и пунктуация сохранены. Напомним, можно будет поставить подпись не более чем за одного кандидата либо одну подпись «против всех».',
   '3 КУРС, 4 ПОТОК (выборы завтра на третей паре в аудитории 128 лабораторного корпуса А)',
   '— ЧЕБОТАЕВА Виктория Алексеевна, 332 группа',
   'Хочу представлять интересы своего потока, участвовать в распределении повышенных стипендий, защищать права студентов, помогать в организации праздников и студентам младших курсов.']}}

## functions for processing of posts about elections

### functions for posts with announcements

In [32]:

# вытаскивает из строки номера курса и потока (в предположении, что они в этой строке есть)

# для обычных потоков
def year_stream_no_by_line(line):
    if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line):
        year_stream_line = (re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line))[0]
        year_stream_line_d = re.sub('\D', '', year_stream_line)
        year_no = int(year_stream_line_d[0])
        stream_no = int(year_stream_line_d[1])
    elif re.findall(r'\D(\d\.\d)\D', line):
        year_stream_line = (re.findall(r'\D(\d\.\d)\D', line))[0]
#         print(year_stream_line)
#         print(year_stream_line[0])
        year_no = year_stream_line[0]
        stream_no = year_stream_line[2]
    
    year_stream_no = str(year_no) + '.' + str(stream_no)
    return year_stream_no, year_no, stream_no

# для магистратуры
def magister_year_by_line(line):
#     если номер года записан цифрами
    if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line))[0]
#         print(year_stream_line)
        year_no = int(year_stream_line[1])
        year_stream_no = 'm' + str(year_no)
#     если номер года записан буквами
    elif re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line))[0]
#         print(year_stream_line)
        year_no = 0
        if str.lower(year_stream_line[1]) == 'перв':
            year_no = 1
        elif str.lower(year_stream_line[1]) == 'втор':
            year_no = 2
        year_stream_no = 'm' + str(year_no)
    return year_stream_no, year_no

# примеры для магистратуры:
# МАГИСТРЫ ПЕРВОГО ГОДА ОБУЧЕНИЯ (выборы завтра на второй паре в аудитории 12-03) +
# МАГИСТРЫ, 2 ГОД ОБУЧЕНИЯ +
# МАГИСТРЫ 2 ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ 1-ГО ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ, 1 КУРС +

# line = 'МАГИСТРЫ, 1 КУРС'
# magister_year_by_line(line)

In [329]:
# line1 = '— ЯНКОВА Маргарита Михайловна' +
# line1 = '— ОСТРОУХОВА Наталья Владимировна, группа 402' 
# line1 = '— ЧЕБОТАЕВА Виктория Алексеевна, 332 группа' +
# line1 = '— ПРОМЫСЛОВ Платон Валерьевич, гр.409' +
# line1 = '— БУН-ЯН-ТУН Владислав Владимирович, группа 125' +
line1 = 'Перейдём к старшим курсам. Есть две новости, хорошая и плохая. Плохая: от механиков 5 курса до сих пор не поступило ни одной заявки. Поторопитесь, не оставляйте свой поток без представителя — время до полуночи ещё есть! Хорошая: на экономпотоке четверокурсников кандидат в наличии, с ним и знакомим.'

# r'\—([\w\s\-]+)[\,\.\s\w]*([\d]*)'

if len(re.findall(r'\—([\w\s\-]+)', line1)) > 0 and re.search(r'\—([\w\s\-]+)', line1).start() == 0:
    name_line = (re.findall(r'\—([\w\s\-]+)', line1))
    print(name_line)
    name = (re.sub('\W\s', '', name_line[0])).strip().title()
    if re.sub('\D', '', line1):
        academic_group_no = int(re.sub('\D', '', line1))
        print(academic_group_no)

print(name)

Бекбулатов Рамзан Ломалиевич


In [330]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО КАНДИДАТОВ (по списку строк с текстом)


# по списку строк с текстом составляем СЛОВАРЬ: имя: курс-поток, группа, номер первой строки в тексте
def stream_nomination_by_list(list_of_lines):
    nominated_dict = {}
#     year_stream_no, year_no, stream_no = '', 0, 0
    
    for i in range(len(list_of_lines)):
#         В НАЧАЛЕ ШАБЛОНА БЫЛО \s (убрал, так как иногда забывают пробел, а алгоритм всё равно жадный)
#         r'[\w\s]+,\sгруппа\s[\d]{3}' - для фиксированного написания группы (и не для маги)
        nomination_pattern = r'\—([\w\s\-]+)'
        if len(re.findall(nomination_pattern, list_of_lines[i])) > 0 and re.search(nomination_pattern, list_of_lines[i]).start() == 0:
            name_line_no = i
            name_line = re.findall(nomination_pattern, list_of_lines[i])
            name = (re.sub('\W\s', '', name_line[0])).strip().title()
            if re.sub('\D', '', list_of_lines[i]):
                academic_group_no = int(re.sub('\D', '', list_of_lines[i]))
            else:
                academic_group_no = ''
            another_one_dict = {}
            another_one_dict[name] = {
                'academic_group_no': academic_group_no, 
                'name_line_no': name_line_no
            }
            nominated_dict = {**nominated_dict, **another_one_dict}
            
    # первый и последний индексы программы кандидата в списке
    name_list = list(nominated_dict.keys())
    start_index_list = [nominated_dict[key]['name_line_no'] for key in list(nominated_dict.keys())]
    last_index_list = [i-1 for i in start_index_list if start_index_list.index(i) > 0]
    last_index_list.append(len(list_of_lines)-1)
    
    dict_of_nominated_upd = {}
    
    for name in name_list:
        dict_of_nominated_upd[name] = [start_index_list[name_list.index(name)], last_index_list[name_list.index(name)]]
    
    # обновить словарь: имя: номер группы, текст программы
    for name in dict_of_nominated_upd:
        program_lines = []

        for k in range(dict_of_nominated_upd[name][0]+1, dict_of_nominated_upd[name][1]+1):
            program_lines.append(list_of_lines[k])
        program_txt = '\n'.join(program_lines)

        nominated_dict[name].update({'program_txt': program_txt})
        del nominated_dict[name]['name_line_no']

    return nominated_dict


# вся инфа про номинации на потоке
def stream_nomination_info_by_dict(list_of_lines):
    year_stream_no = ''
    year_no, stream_no = 0, 0
    
    for i in range(len(list_of_lines)):
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
                
    dict_nominations = stream_nomination_by_list(list_of_lines)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'dict_nominations': dict_nominations
    }
    
    return dict_stream_elections



### functions for posts with results

In [352]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО РЕЗУЛЬТАТЫ ВЫБОРОВ (по списку строк с текстом)

# ДОБАВИТЬ МАГИСТРАТУРУ

# по списку строк с кандидатами составляем СЛОВАРЬ: имя: голоса, статус
def stream_election_result_by_list(nominated_with_votes):
    dict_nominated = {}

    for line in nominated_with_votes:
        kek = (re.findall(r'[\—\s]*([\w\-\s]+)[\:\;]\s([\d]+)\sголос', line))[0]
        name = kek[0].strip().title()
        votes_cnt = kek[1]
#         name = ((re.findall(r'[\w\s]+', line))[0]).strip().title()
#         votes_line = (re.findall(r'\s[\d]+\sголос', line))[0]
#         votes_cnt = int(re.sub('\D', '', votes_line))
        dict_nominated[name] = {'votes_cnt': votes_cnt}
        if (re.findall(r' избран', line)) and not (re.findall(r' не избран', line)):
            elected_flg = 1
        else:
            elected_flg = 0
        dict_nominated[name].update({'elected_flg': elected_flg})
    return dict_nominated


# вся инфа про результаты выборов на потоке
def stream_election_info_by_dict(list_of_lines):
    nominated_with_votes = []
    year_stream_no = ''
    year_no, stream_no = 0, 0
    voted_cnt, stream_size, invalid_cnt, NOTA_cnt = 0, 0, 0, 0
    
    for i in range(len(list_of_lines)):
                    
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]) or re.findall(r'\D(\d\.\d)\D', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
            
        if re.findall('Явка: ', list_of_lines[i]):
            turnout_line = (re.findall(r'[\d]+\s[\w]+\sиз\s[\d]+', list_of_lines[i]))[0]
            voted_cnt = int(((re.findall(r'[\d]+\s', turnout_line))[0]).strip())
            stream_size = int(((re.findall(r'\s[\d]+', turnout_line))[0]).strip())
            
        if len(re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i])) == 0: # отдельно, т.к. может такой записи не быть
            pass
        else:
            invalid_line_no = i
            invalid_line = (re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i]))[0]
            invalid_cnt = re.sub('\D', '', invalid_line)
            
        if re.findall('Против всех', list_of_lines[i]):
            NOTA_line_no = i
            NOTA_line = (re.findall(r'\s[\d]+\sголос', list_of_lines[i]))[0]
            NOTA_cnt = re.sub('\D', '', NOTA_line)
            
        if re.findall('[\—\s]*([\w\-\s]+)[\:\;]\s([\d]+)\sголос', list_of_lines[i]) and not (re.findall('Против всех', list_of_lines[i])):
            nominated_with_votes.append(list_of_lines[i])
            
        if re.findall('Подтверждено ответственн', list_of_lines[i]):
            organizer_line = (re.findall(r'за\sвыборы\s[\w\s,]+', list_of_lines[i]))[0]
            organizer_line = organizer_line[10::]
            organizer_list = re.split(', | и ',organizer_line)
    
    dict_nominated_with_votes = stream_election_result_by_list(nominated_with_votes)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'stream_size': stream_size, 
        'voted_cnt': voted_cnt,
        'invalid_cnt': invalid_cnt,
        'NOTA_cnt': NOTA_cnt,
        'organizers': organizer_list,
        'dict_nominated_with_votes': dict_nominated_with_votes
    }
    
    return dict_stream_elections


## Processing of all election posts for 4-11 convocations

### launching

In [373]:
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С АНОНСАМИ ВЫБОРОВ
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С РЕЗУЛЬТАТАМИ ВЫБОРОВ

from itertools import chain

announcement_links = {}
result_links = {}

for num in chain(
#     range(6660, 6720) #, # test
#     range(2200, 2360) #, # созыв 4 +
#     range(3140, 3530) #, # созыв 5 +
#     range(5040, 5360) #, # созыв 6 +
#     range(6540, 6720) #, # созыв 7 
    range(7470, 7780), # созыв 8 
#     range(8240, 8420), # созыв 9 
#     range(9100, 9400), # созыв 10 
#     range(9800, 10040) # созыв 11 
):
    link = 'https://vk.com/wall-33327655_' + str(num)

    test_dict = vk_post_info_by_link(link, 'georgian')
    
#     поправка из-за коммента с доп.кандидатами
    if 'https://vk.com/wall-33327655_2232' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_2232']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_2232']['content_subtype'] = 'announcement'
#     поправка из-за коммента с доп.кандидатами
    if 'https://vk.com/wall-33327655_2305' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_2305']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_2305']['content_subtype'] = 'announcement'
#     поправка из-за отсутствия кодовой фразы
    if 'https://vk.com/wall-33327655_2230' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_2230']['content_subtype'] = 'announcement'
#     поправка из-за отсутствия выборного хэштега
    if 'https://vk.com/wall-33327655_7509' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_7509']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_7509']['content_subtype'] = 'result'
#     поправка из-за отсутствия кодовой фразы
    if 'https://vk.com/wall-33327655_7657' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_7657']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_7657']['content_subtype'] = 'announcement'
    
    print('{0} : {1} \t\t/ {2} \t\t/ {3}'.format(link, test_dict[link]['link_type'], test_dict[link]['content_type'], test_dict[link]['content_subtype']))

    for d in test_dict.values():
        if d['content_type'] == 'election' and d['content_subtype'] == 'announcement':
            announcement_links = {**announcement_links, **test_dict}
        if d['content_type'] == 'election' and d['content_subtype'] == 'result':
            result_links = {**result_links, **test_dict}


for link in announcement_links:
    announcement_links[link].update({'nominated_info': stream_nomination_info_by_dict(announcement_links[link]['list_of_lines'])})
    del announcement_links[link]['list_of_lines']
    
for link in result_links:
    result_links[link].update({'stream_election_info': stream_election_info_by_dict(result_links[link]['list_of_lines'])})
    del result_links[link]['list_of_lines']

# print(announcement_links)
# print(result_links)

# поправки из-за единичных нестандартных случаев

if 'https://vk.com/wall-33327655_2232' in announcement_links.keys():
    announcement_links['https://vk.com/wall-33327655_2232']['nominated_info']['year_stream_no'] = '1.2'
    announcement_links['https://vk.com/wall-33327655_2232']['nominated_info']['year_no'] = 1
    announcement_links['https://vk.com/wall-33327655_2232']['nominated_info']['stream_no'] = 2
if 'https://vk.com/wall-33327655_2305' in announcement_links.keys():
    announcement_links['https://vk.com/wall-33327655_2305']['nominated_info']['year_stream_no'] = '3.1'
    announcement_links['https://vk.com/wall-33327655_2305']['nominated_info']['year_no'] = 3
    announcement_links['https://vk.com/wall-33327655_2305']['nominated_info']['stream_no'] = 1
if 'https://vk.com/wall-33327655_7695' in result_links.keys():
    result_links['https://vk.com/wall-33327655_7695']['stream_election_info']['year_stream_no'] = 'm1'
    result_links['https://vk.com/wall-33327655_7695']['stream_election_info']['year_no'] = 1
if 'https://vk.com/wall-33327655_9205' in result_links.keys():
    result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['year_stream_no'] = '6.3'
    result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['year_no'] = 6
    result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['stream_no'] = 3
if 'https://vk.com/wall-33327655_9267' in result_links.keys():
    result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['year_stream_no'] = '6.1'
    result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['year_no'] = 6
    result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['stream_no'] = 1



https://vk.com/wall-33327655_7470 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7471 : post 		/ text 		/ other
https://vk.com/wall-33327655_7472 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7473 : repost 		/ text 		/ other
https://vk.com/wall-33327655_7474 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7475 : repost 		/ text 		/ other
https://vk.com/wall-33327655_7476 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7477 : post 		/ election 		/ other
https://vk.com/wall-33327655_7478 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7479 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_7480 : comment 		/ text 		/ other
https://vk.com/wall-33327655_7481 : comment 		/ text 		/ other
https://vk.com/wall-33327655_7482 : comment 		/ text 		/ other
https://vk.com/wall-33327655_7483 : comment 		/ text 		/ other
https://vk.com/wall-33327655_7484 : comment 		/ text 		/ other
h

https://vk.com/wall-33327655_7597 : post 		/ text 		/ other
https://vk.com/wall-33327655_7598 : comment 		/ text 		/ other
https://vk.com/wall-33327655_7599 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7600 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7601 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7602 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_7603 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7604 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7605 : post 		/ election 		/ other
https://vk.com/wall-33327655_7606 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_7607 : comment 		/ text 		/ other
https://vk.com/wall-33327655_7608 : post 		/ election 		/ other
https://vk.com/wall-33327655_7609 : post 		/ election 		/ other
https://vk.com/wall-33327655_7610 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_7611 : comment 

https://vk.com/wall-33327655_7720 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7721 : repost 		/ text 		/ other
https://vk.com/wall-33327655_7722 : repost 		/ text 		/ other
https://vk.com/wall-33327655_7723 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7724 : post 		/ text 		/ other
https://vk.com/wall-33327655_7725 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7726 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7727 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7728 : post 		/ election 		/ result
https://vk.com/wall-33327655_7729 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7730 : post 		/ text 		/ other
https://vk.com/wall-33327655_7731 : repost 		/ text 		/ other
https://vk.com/wall-33327655_7732 : repost 		/ text 		/ other
https://vk.com/wall-33327655_7733 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_7734 : comment 		/ text 		/

In [262]:
# announcement_streams['https://vk.com/wall-33327655_7518']

### creating tables about candidates and streams

In [374]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

announcement_streams = {}
for link in announcement_links.keys():
    announcement_streams[link] = {
        'year_x_year_stream': str(announcement_links[link]['link_year_no']) + '_' + announcement_links[link]['nominated_info']['year_stream_no'],
        'link_date': announcement_links[link]['link_date'],
        'link_year_no': announcement_links[link]['link_year_no'], 
        'link_month_no': announcement_links[link]['link_month_no'], 
        'link_day_no': announcement_links[link]['link_day_no'], 
        'link_time': announcement_links[link]['link_time'], 
        'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
        'year_no': announcement_links[link]['nominated_info']['year_no'], 
        'stream_no': announcement_links[link]['nominated_info']['stream_no']
    }
# announcement_streams

announcement_streams_df =  pd.DataFrame(announcement_streams.items(), columns=['link', 'params'])
announcement_streams_df_2 = pd.concat([announcement_streams_df.drop(['params'], axis=1), announcement_streams_df['params'].apply(pd.Series)], axis=1)
announcement_streams_df_3 = announcement_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
announcement_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no
0,https://vk.com/wall-33327655_7621,2018_1.1,2018-10-23,2018,10,23,0:00,1.1,1,1
1,https://vk.com/wall-33327655_7479,2018_1.2,2018-10-12,2018,10,12,21:45,1.2,1,2
2,https://vk.com/wall-33327655_7645,2018_1.3,2018-10-23,2018,10,23,23:38,1.3,1,3
3,https://vk.com/wall-33327655_7622,2018_2.1,2018-10-23,2018,10,23,0:03,2.1,2,1
4,https://vk.com/wall-33327655_7644,2018_2.3,2018-10-23,2018,10,23,23:19,2.3,2,3
5,https://vk.com/wall-33327655_7619,2018_3.1,2018-10-21,2018,10,21,23:45,3.1,3,1
6,https://vk.com/wall-33327655_7657,2018_3.2,2018-10-25,2018,10,25,0:10,3.2,3,2
7,https://vk.com/wall-33327655_7552,2018_3.3,2018-10-17,2018,10,17,23:27,3.3,3,3
8,https://vk.com/wall-33327655_7623,2018_3.4,2018-10-23,2018,10,23,0:06,3.4,3,4
9,https://vk.com/wall-33327655_7667,2018_4.1,2018-10-25,2018,10,25,21:44,4.1,4,1


In [ ]:
# !!!! РАЗОБРАТЬСЯ С ПОВТОРНЫМИ ПОСТАМИ ВНУТРИ ОДНОГО ПОТОКА


In [363]:
announcement_streams_df_3.to_csv('ssmm_announcement_info_conv6.csv', sep='\t', encoding='utf-8', index=False)

In [ ]:
'''
БАГ : https://vk.com/wall-33327655_7621
скурихин пишет программу по пунктам с длинного тире, НАДО ОГРАНИЧИТЬ ЧИСЛО СЛОВ В ШАБЛОНЕ
'''

In [375]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

announcement_names = {}
for link in announcement_links.keys():
    for name in announcement_links[link]['nominated_info']['dict_nominations'].keys():
        announcement_names[name] = {
            'link_year_no_x_name': str(announcement_links[link]['link_year_no']) + "_" + "_".join(name.split()),
            'link_year_no': announcement_links[link]['link_year_no'], 
            'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
            'year_no': announcement_links[link]['nominated_info']['year_no'], 
            'stream_no': announcement_links[link]['nominated_info']['stream_no'], 
            'academic_group_no': announcement_links[link]['nominated_info']['dict_nominations'][name]['academic_group_no'], 
            'program_txt': announcement_links[link]['nominated_info']['dict_nominations'][name]['program_txt'],
            'link': link
        }
# announcement_names

announcement_names_df =  pd.DataFrame(announcement_names.items(), columns=['name', 'params'])
announcement_names_df_2 = pd.concat([announcement_names_df.drop(['params'], axis=1), announcement_names_df['params'].apply(pd.Series)], axis=1)
announcement_names_df_2[:60]
# .head(10)


,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link
0,Шерстобитов Андрей Сергеевич,2018_Шерстобитов_Андрей_Сергеевич,2018,1.2,1,2,107,"""Планирую отстаивать права студентов и влиять ...",https://vk.com/wall-33327655_7479
1,Алиев Рамиль Камиль Оглы,2018_Алиев_Рамиль_Камиль_Оглы,2018,1.2,1,2,107,"""Я как активный гражданин не мог пройти мимо С...",https://vk.com/wall-33327655_7479
2,Устинов Никита Евгеньевич,2018_Устинов_Никита_Евгеньевич,2018,1.2,1,2,108,"""Хочу принимать участие в организации мероприя...",https://vk.com/wall-33327655_7479
3,Сиухина Арина Алексеевна,2018_Сиухина_Арина_Алексеевна,2018,1.2,1,2,112,"""Hey! My name is Arina. I wasn’t taught to wri...",https://vk.com/wall-33327655_7479
4,Логинова Ирина Игоревна,2018_Логинова_Ирина_Игоревна,2018,1.2,1,2,109,"""Я бы хотела сделать жизнь студентов на мехмат...",https://vk.com/wall-33327655_7479
5,Ефимкина Арина Вячеславовна,2018_Ефимкина_Арина_Вячеславовна,2018,m1,1,0,,"""Обожаю организаторскую и творческую деятельно...",https://vk.com/wall-33327655_7518
6,Вихорева Надежда Дмитриевна,2018_Вихорева_Надежда_Дмитриевна,2018,3.3,3,3,323,"""Привет!\nТы можешь об этом и не догадываться,...",https://vk.com/wall-33327655_7552
7,Кудасов Матвей Джамалханович,2018_Кудасов_Матвей_Джамалханович,2018,4.3,4,3,426,"""Всем привет. В предыдущем созыве я был ответс...",https://vk.com/wall-33327655_7602
8,Денисов Илья Олегович,2018_Денисов_Илья_Олегович,2018,6.2,6,2,608,""" ""Утром мажу бутерброд -\nСразу мысль: а как ...",https://vk.com/wall-33327655_7606
9,Тонконогов Александр Владиславович,2018_Тонконогов_Александр_Владиславович,2018,6.1,6,1,601,"""Добрый день!\nМеня зовут Александр, и я, как ...",https://vk.com/wall-33327655_7610


In [ ]:
# присвоить ssmm_id в порядке публикации постов (дата, время)
# ИЛИ другим способом ОТСОРТИРОВАТЬ КАНДИДАТОВ (сейчас они в порядке появления в постах в порядке выхода постов, вроде норм)

In [366]:
announcement_names_df_2.to_csv('ssmm_candidates_info_conv6.csv', sep='\t', encoding='utf-8', index=False)

 ### creating tables about results

In [ ]:
'''
БАГ:
- https://vk.com/wall-33327655_7695 - не считывается поток 
'''

In [220]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

result_streams = {}
for link in result_links.keys():
    result_streams[link] = {
        'year_x_year_stream': str(result_links[link]['link_year_no']) + '_' + result_links[link]['stream_election_info']['year_stream_no'],
        'link_date': result_links[link]['link_date'],
        'link_year_no': result_links[link]['link_year_no'], 
        'link_month_no': result_links[link]['link_month_no'], 
        'link_day_no': result_links[link]['link_day_no'], 
        'link_time': result_links[link]['link_time'], 
        'year_stream_no': result_links[link]['stream_election_info']['year_stream_no'], 
        'year_no': result_links[link]['stream_election_info']['year_no'], 
        'stream_no': result_links[link]['stream_election_info']['stream_no'], 
        'stream_size': result_links[link]['stream_election_info']['stream_size'], 
        'voted_cnt': result_links[link]['stream_election_info']['voted_cnt'], 
        'invalid_cnt': result_links[link]['stream_election_info']['invalid_cnt'], 
        'NOTA_cnt': result_links[link]['stream_election_info']['NOTA_cnt'], 
        'organizers': result_links[link]['stream_election_info']['organizers']
    }
# announcement_streams

result_streams_df =  pd.DataFrame(result_streams.items(), columns=['link', 'params'])
result_streams_df_2 = pd.concat([result_streams_df.drop(['params'], axis=1), result_streams_df['params'].apply(pd.Series)], axis=1)
result_streams_df_3 = result_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
result_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers
0,https://vk.com/wall-33327655_7676,2018_1.1,2018-10-26,2018,10,26,17:00,1.1,1,1,168,117,7,4,"[Георгием Малиновским, Олегом Бузенковым]"
1,https://vk.com/wall-33327655_7509,2018_1.2,2018-10-14,2018,10,14,21:00,1.2,1,2,151,104,2,8,"[Георгием Малиновским, Кириллом Филатовым]"
2,https://vk.com/wall-33327655_7685,2018_1.3,2018-10-27,2018,10,27,14:00,1.3,1,3,123,98,3,5,"[Сенотовым Валерием, Малиновским Георгием]"
3,https://vk.com/wall-33327655_7677,2018_2.1,2018-10-26,2018,10,26,19:00,2.1,2,1,134,81,5,1,"[Валерией Фиткович, Николаем Малаховым]"
4,https://vk.com/wall-33327655_7684,2018_2.3,2018-10-27,2018,10,27,12:00,2.3,2,3,107,75,1,4,"[Сенотовым Валерием, Малиновским Георгием]"
5,https://vk.com/wall-33327655_7675,2018_3.1,2018-10-26,2018,10,26,14:00,3.1,3,1,107,58,3,11,"[Валерием Сенотовым, Александром Тонконоговым]"
6,https://vk.com/wall-33327655_7728,2018_3.2,2018-11-12,2018,11,12,22:56,3.2,3,2,94,49,1,4,"[Ильёй Денисовым, Дарьей Волковой]"
7,https://vk.com/wall-33327655_7670,2018_3.3,2018-10-25,2018,10,25,23:52,3.3,3,3,103,55,0,8,"[Валерием Сенотовым, Александром Тонконоговым]"
8,https://vk.com/wall-33327655_7681,2018_3.4,2018-10-27,2018,10,27,10:00,3.4,3,4,40,25,0,0,"[Валерием Сенотовым, Александром Тонконоговым]"
9,https://vk.com/wall-33327655_7694,2018_4.1,2018-11-02,2018,11,2,22:25,4.1,4,1,86,45,2,11,"[Денисовым Ильей, Воробьевой Наталией]"


In [221]:
result_streams_df_3.to_csv('ssmm_results_info_conv8.csv', sep='\t', encoding='utf-8', index=False)

In [222]:
# ОТДЕЛЬНАЯ ТАБЛИЦА ПО ОРГАНИЗАТОРАМ

organizers_df = result_streams_df_3[['year_x_year_stream','organizers']].apply(lambda x: x.explode()).reset_index().drop(['index'], axis=1)
# organizers_df


In [223]:
organizers_df.to_csv('ssmm_organizers_info_conv8.csv', sep='\t', encoding='utf-8', index=False)

In [367]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

result_names = {}
for link in result_links.keys():
    for name in result_links[link]['stream_election_info']['dict_nominated_with_votes'].keys():
        result_names[name] = {
            'link_year_no_x_name': str(result_links[link]['link_year_no']) + "_" + "_".join(name.split()),
            'link_year_no': result_links[link]['link_year_no'], 
            'year_stream_no': result_links[link]['stream_election_info']['year_stream_no'], 
            'year_no': result_links[link]['stream_election_info']['year_no'], 
            'stream_no': result_links[link]['stream_election_info']['stream_no'], 
            'votes_cnt': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['votes_cnt'],
            'elected_flg': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['elected_flg'],
            'link': link
        }

result_names_df =  pd.DataFrame(result_names.items(), columns=['name', 'params'])
result_names_df_2 = pd.concat([result_names_df.drop(['params'], axis=1), result_names_df['params'].apply(pd.Series)], axis=1)
result_names_df_2
# .head(10)


,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,votes_cnt,elected_flg,link
0,Титов Максим Сергеевич,2016_Титов_Максим_Сергеевич,2016,1.3,1,3,32,0,https://vk.com/wall-33327655_5088
1,Кудрявцев Сергей Сергеевич,2016_Кудрявцев_Сергей_Сергеевич,2016,1.3,1,3,36,0,https://vk.com/wall-33327655_5088
2,Поликушин Владимир Николаевич,2016_Поликушин_Владимир_Николаевич,2016,1.3,1,3,40,1,https://vk.com/wall-33327655_5088
3,Штерн Анастасия Георгиевна,2016_Штерн_Анастасия_Георгиевна,2016,1.3,1,3,48,1,https://vk.com/wall-33327655_5088
4,Корж Даниил Сергеевич,2016_Корж_Даниил_Сергеевич,2016,1.3,1,3,52,1,https://vk.com/wall-33327655_5088
...,...,...,...,...,...,...,...,...,...
56,Садова Карина Александровна,2016_Садова_Карина_Александровна,2016,4.1,4,1,18,1,https://vk.com/wall-33327655_5357
57,Семенюк Павел Игоревич,2016_Семенюк_Павел_Игоревич,2016,4.1,4,1,25,1,https://vk.com/wall-33327655_5357
58,Тонконогов Александр Владиславович,2016_Тонконогов_Александр_Владиславович,2016,4.1,4,1,23,1,https://vk.com/wall-33327655_5357
59,Бахметьева Марина Игоревна,2016_Бахметьева_Марина_Игоревна,2016,4.1,4,1,11,0,https://vk.com/wall-33327655_5357


In [368]:
result_names_df_2.to_csv('ssmm_votes_info_conv6.csv', sep='\t', encoding='utf-8', index=False)

In [369]:
# full table for 4-11 convocations:

def united_table_for_4_11(name):
    list_of_df = []
    for i in range(4,11+1):
        df = pd.read_csv('ssmm_'+name+'_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
        list_of_df.append(df)
    kek = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
    kek.to_csv('ssmm_'+name+'_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)
    return None

names = ['announcement','candidates','results','organizers','votes']

for name in names:
    united_table_for_4_11(name)

## Completeness testing

In [ ]:
# ПОПРИДЖОЙНИВАТЬ АНОНСЫ НА РЕЗУЛЬТАТЫ, ЧТОБЫ ПОНЯТЬ, КАКИЕ ПОСТЫ С РЕЗУЛЬТАТАМИ НЕ РАСПОЗНАЛИСЬ

# - по созывам, внутри созывов:
#     - по потокам
#     - по кандидатам

In [370]:
ssmm_announcements_info = pd.read_csv('ssmm_announcement_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_candidates_info = pd.read_csv('ssmm_candidates_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_results_info = pd.read_csv('ssmm_results_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_organizers_info = pd.read_csv('ssmm_organizers_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_votes_info = pd.read_csv('ssmm_votes_info_conv_4_11.csv', sep='\t', encoding='utf-8')


In [234]:
list(ssmm_candidates_info)

['name',
 'link_year_no_x_name',
 'link_year_no',
 'year_stream_no',
 'year_no',
 'stream_no',
 'academic_group_no',
 'program_txt',
 'link']

In [235]:
list(ssmm_votes_info)

['name',
 'link_year_no_x_name',
 'link_year_no',
 'year_stream_no',
 'year_no',
 'stream_no',
 'votes_cnt',
 'elected_flg',
 'link']

In [233]:
df1 = ssmm_announcements_info[['year_x_year_stream','link']]
df2 = ssmm_results_info[['year_x_year_stream','link']]

df1_x_d2 = df1.merge(df2, left_on='year_x_year_stream', right_on='year_x_year_stream',how='outer')
# df1_x_d2[120:180]

# всё ок, кроме:
# 2014_m2 - нет в ssmm_announcements_info - не было анонса


In [244]:
ssmm_announcements_info[ssmm_announcements_info['link'] == 'https://vk.com/wall-33327655_2218']

,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no
12,https://vk.com/wall-33327655_2218,2014_3.4,2014-11-18,2014,11,18,1:42,3.4,3,4


In [245]:
ssmm_candidates_info[ssmm_candidates_info['link'] == 'https://vk.com/wall-33327655_2218']

,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link


In [247]:
ssmm_candidates_info[ssmm_candidates_info['link'] == 'https://vk.com/wall-33327655_7518']

,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link


In [372]:
df1 = ssmm_candidates_info[['link_year_no_x_name','link']]
df2 = ssmm_votes_info[['link_year_no_x_name','link']]

df1_x_d2 = df1.merge(df2, left_on='link_year_no_x_name', right_on='link_year_no_x_name',how='outer')
df1_x_d2_nan = df1_x_d2[df1_x_d2['link_x'].isna()]
# df1_x_d2_nan = df1_x_d2[df1_x_d2['link_y'].isna()]
# df1_x_d2_nan = df1_x_d2[df1_x_d2[kkk].isna() for any(kkk in ['link_x', 'link_y'])]
# df1_x_d2_nan = df1_x_d2[df1_x_d2.isnull()[['link_x', 'link_y']].any()]
df1_x_d2_nan
# df1_x_d2[120:180]

# ПРОБЛЕМЫ С МАГИСТРАТУРОЙ ИЗ_ЗА ОТСУТСТВИЯ ШАБЛОНА В stream_nomination_by_list
# ПРОБЛЕМЫ С РАЗЛИЧНЫМ НАПИСАНИЕМ ИМЕН (Е/Ё, опечатки)
# ПРОБЛЕМЫ С ШАБЛОНОМ В РАЙОНЕ СЛОВО ГРУППА
# ПРОБЛЕМЫ С ШАБЛОНОМ ИЗ_ЗА СОСТАВНЫХ ФАМИЛИЙ


# 2016_Думаревский_Александр_Дмитриевич	 - нет в ssmm_votes_info (есть https://vk.com/wall-33327655_5144) - СНЯЛСЯ
# 2016_Сакаев_Роберт_Марселевич	 - нет в ssmm_votes_info (есть https://vk.com/wall-33327655_5313) - Марселевич/Викторович (правильно Марселевич)
# 2016_Рогачёв_Александр_Викторович	 - нет в ssmm_votes_info (есть https://vk.com/wall-33327655_5320) - Ё/Е

# 163	2017_Порфирьева_Наталия_Алексеевна	https://vk.com/wall-33327655_6581	NaN - НаталЬя/НаталИя (правильно: хз)
# 167	2017_Третьяков_Артем_Алексеевич	https://vk.com/wall-33327655_6586	NaN - Ё/Е
# 168	2017_Муравьёва_Анна_Сергеевна	https://vk.com/wall-33327655_6586	NaN - Ё/Е

# 227	2019_Муравьева_Анна_Сергеевна	https://vk.com/wall-33327655_8267	NaN - Ё/Е
# 248	2019_Воронин_Вадим_Юрьевич	https://vk.com/wall-33327655_8323	NaN - СНЯЛСЯ
# 253	2019_Лобанова_Елена_Валерьевна	https://vk.com/wall-33327655_8328	NaN - Ё/Е

# 297	2020_Пушкарёва_Анна_Константиновна	https://vk.com/wall-33327655_9228	NaN - Ё/Е
# 309	2020_Илизаров_Год_Эмануилович	https://vk.com/wall-33327655_9251	NaN - иЛЛизаров/иЛизаров (правильно: Л)

# 344	2021_Витковский_Андрей_Владиславович	https://vk.com/wall-33327655_9894	NaN - случайный ЭНТЕР (https://vk.com/wall-33327655_10007)


# 367	2014_Вергазов_Марат_Марсович	NaN	https://vk.com/wall-33327655_2313 - НЕ БЫЛО АНОНСА

# 379	2016_Рогачев_Александр_Викторович	NaN	https://vk.com/wall-33327655_5344 - Ё/Е
# 380	2016_Сакаев_Роберт_Викторович	NaN	https://vk.com/wall-33327655_5350 - Марселевич/Викторович (правильно Марселевич)

# 381	2018_Шишкин_Игнат_Владимирович	NaN	https://vk.com/wall-33327655_7685 - ИЗ_ЗА ПАРНОЙ ЗАЯВКИ https://vk.com/wall-33327655_8330
# 382	2018_Ефимкина_Арина_Вячеславовна	NaN	https://vk.com/wall-33327655_7695 - МАГА
# 383	2018_Антонов_Иван_Андреевич	NaN	https://vk.com/wall-33327655_7728 - СОКРАЩЕНИЕ ДО "гр."/"гр. " вместо "группа"
# 384	2018_Дементьев_Фёдор_Андреевич	NaN	https://vk.com/wall-33327655_7728 - СОКРАЩЕНИЕ ДО "гр."/"гр. " вместо "группа"
# 385	2018_Панина_Татьяна_Андреевна	NaN	https://vk.com/wall-33327655_7741 - СОКРАЩЕНИЕ ДО "гр."/"гр. " вместо "группа"
# 386	2018_Промыслов_Платон_Валерьевич	NaN	https://vk.com/wall-33327655_7741 - СОКРАЩЕНИЕ ДО "гр."/"гр. " вместо "группа"

# 387	2019_Аскарова_Айжамал_Шамшетдиновна	NaN	https://vk.com/wall-33327655_8372 - МАГА
# 388	2019_Лобанова_Алёна_Валерьевна	NaN	https://vk.com/wall-33327655_8391 - Алёна/Елена (правильно: ?)
# 389	2019_Муравьёва_Анна_Сергеевна	NaN	https://vk.com/wall-33327655_8396 - Ё/Е
# 390	2019_Янкова_Маргарита_Михайловна	NaN	https://vk.com/wall-33327655_8408 - МАГА

# 391	2020_Янкова_Маргарита_Михайловна	NaN	https://vk.com/wall-33327655_9310 - МАГА
# 392	2020_Кирюхина_Алёна_Андреевна	NaN	https://vk.com/wall-33327655_9321 - "ХХХ группа" вместо "группа ХХХ" (https://vk.com/wall-33327655_9216)
# 393	2020_Пушкарева_Анна_Константиновна	NaN	https://vk.com/wall-33327655_9340 - Ё/Е
# 394	2020_Иллизаров_Год_Эмануилович	NaN	https://vk.com/wall-33327655_9342 - иЛЛизаров/иЛизаров (правильно: Л)

# 395	2021_Янушкевич_Иван_Михайлович	NaN	https://vk.com/wall-33327655_9984 - "ХХХ группа" вместо "группа ХХХ"
# 396	2021_Гаспарян_Иосиф_Жорович	NaN	https://vk.com/wall-33327655_9989 - "ХХХ группа" вместо "группа ХХХ"
# 397	2021_Каляев_Тимур_Джанбулатович	NaN	https://vk.com/wall-33327655_10018 - МАГА
# 398	2021_Маммагаджиев_Руслан_Гаджимагомедович	NaN	https://vk.com/wall-33327655_10018 - МАГА
# 399	2021_Третьякова_Екатерина_Алексеевна	NaN	https://vk.com/wall-33327655_10018 - МАГА


,link_year_no_x_name,link_x,link_y
375,2014_Вергазов_Марат_Марсович,NaN,https://vk.com/wall-33327655_2313
376,2016_Рогачев_Александр_Викторович,NaN,https://vk.com/wall-33327655_5344
377,2016_Сакаев_Роберт_Викторович,NaN,https://vk.com/wall-33327655_5350
378,2017_Порфирьева_Наталья_Алексеевна,NaN,https://vk.com/wall-33327655_6624
379,2017_Третьяков_Артём_Алексеевич,NaN,https://vk.com/wall-33327655_6625
380,2017_Муравьева_Анна_Сергеевна,NaN,https://vk.com/wall-33327655_6625
381,2018_Шишкин_Игнат_Владимирович,NaN,https://vk.com/wall-33327655_7685
382,2018_Ефимкина_Арина_Вячеславовна,NaN,https://vk.com/wall-33327655_7695
383,2018_Антонов_Иван_Андреевич,NaN,https://vk.com/wall-33327655_7728
384,2018_Дементьев_Фёдор_Андреевич,NaN,https://vk.com/wall-33327655_7728
